In [1]:
import json
from os.path import join
import os
import sys
import pickle
import gc
import numpy as np

module_path = os.path.abspath(os.path.join('.','pubmed_data'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pubmed_helper as ph


In [2]:
bioASQ_data_path = "/backup/BioASQ-training7b/"
bioASQ_data_test = json.load(open(os.path.join(bioASQ_data_path,"7b_test_split.json")))

In [3]:
MODE = "regex_full_tokens"

full_tk = ph.load_tokenizer(MODE)

Load regex_full_tokens_tokenizer.p


In [4]:
bioASQ_test_queries_tokenized = full_tk.texts_to_sequences(map(lambda x:x["body"],bioASQ_data_test))


In [5]:
min_token_index = min(map(lambda x:max(x),bioASQ_test_queries_tokenized))
token = full_tk.index_word[min_token_index]

full_tk.word_counts[token]

662370

In [18]:
#choose all tokens with less 700k
max_count = 700000
from keras_new_text import Tokenizer, regex_alfanum_tokenizer

tk = Tokenizer(tokenizer=regex_alfanum_tokenizer)
tk.document_count = 18824355
tk.word_counts = {k:v for k,v in full_tk.word_counts.items() if v<=max_count}

wcounts = list(tk.word_counts.items())
wcounts.sort(key=lambda x: x[1], reverse=True)
# forcing the oov_token to index 1 if it exists
if tk.oov_token is None:
    sorted_voc = []
else:
    sorted_voc = [tk.oov_token]
sorted_voc.extend(wc[0] for wc in wcounts)

# note that index 0 is reserved, never assigned to an existing word
tk.word_index = dict(
    list(zip(sorted_voc, list(range(1, len(sorted_voc) + 1)))))

tk.index_word = dict((c, w) for w, c in tk.word_index.items())

full_tk_voc = set(full_tk.word_counts.keys())
tk_voc = set(tk.word_counts.keys())
diff_voc = full_tk_voc - tk_voc

tk.word_docs = { k:v for k,v in full_tk.word_docs.items() if k not in diff_voc}
for w, c in list(tk.word_docs.items()):
    tk.index_docs[tk.word_index[w]] = c


In [19]:
#verify
bioASQ_test_queries_tokenized_2 = tk.texts_to_sequences(map(lambda x:x["body"],bioASQ_data_test))

any([len(x)==0 for x in bioASQ_test_queries_tokenized_2])

False

In [17]:
bioASQ_data_test[12]["body"]

"Which are the common symptoms of Cushing's syndrome?"

[1368, 11, 1238942]

In [20]:
with open("/backup/pubmed_tokenizers/regex_less_700k_freq_tokenizer.p","wb") as f:
    pickle.dump(tk,f)